Hier werden die Portfolio-Methoden angewandt und mithilfe der Performance-Metriken bewertet. Die Ergebnisse werden in CSV-Dateien festgehalten. Das Akquirieren, modifizieren und beschreiben der Daten befindet sich der Übersicht halber im Notebook Datenbasis. Die Daten werden dort als CSV exportiert und hier als Dataframe wieder eingelesen.

Die Korrektheit der implementierten Methoden und Metriken wird in dem separaten Notebook Verifizierung anhand eines kleinen Teilbestandes aufgezeigt, siehe dazu auch beigefügte Excel-Dateien, in denen die Portfolio-Gewichte "per Hand" nachgerechnet und mit den Ergebnissen aus der Python-Implementierung abgeglichen werden.

ACHTUNG: LANGE LAUFZEIT!

## Importe

In [1]:
#Import der notwendigen Bibliotheken, Installation mit pip bereits erfolgt

import yfinance as yf
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np
import csv
from scipy.optimize import minimize
import math
import random
import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import linkage, dendrogram
import riskfolio as rp
from sklearn.covariance import LedoitWolf
from scipy.spatial.distance import squareform
from riskfolio import HCPortfolio as hc
from pandas.tseries.offsets import MonthBegin
from pandas.tseries.offsets import MonthEnd

## Einlesen der Daten

In [2]:
#Bearbeitete JSE-Daten (Gesamtbestand)

df_JSE = pd.read_csv(
    'WhartonFinal_neu.csv',
    sep=',',
    encoding='utf-8',
    index_col=0,
    decimal = '.',
    low_memory=False,
    parse_dates=['Date'],
    dayfirst=True
)

In [3]:
df_JSE.info

<bound method DataFrame.info of                  8303  15604   15618  16179  16183  16291   16305  16355  \
Date                                                                       
2005-06-01  39.000000  62.11   74.50  14.50   8.60   4.42   82.40   9.95   
2005-06-02  38.800000  62.00   75.00  14.51   8.51   4.40   82.50   9.80   
2005-06-03  37.903333  60.79   75.64  14.50   8.50   4.37   82.61   9.90   
2005-06-06  37.006667  60.75   78.49  13.85   8.62   4.38   82.90   9.90   
2005-06-07  36.110000  60.90   77.90  13.89   8.67   4.43   83.45   9.90   
...               ...    ...     ...    ...    ...    ...     ...    ...   
2025-05-26        NaN    NaN  256.98    NaN  12.95    NaN  168.45    NaN   
2025-05-27        NaN    NaN  255.97    NaN  13.08    NaN  168.48    NaN   
2025-05-28        NaN    NaN  257.02    NaN  13.10    NaN  173.00    NaN   
2025-05-29        NaN    NaN  259.83    NaN  13.32    NaN  175.79    NaN   
2025-05-30        NaN    NaN  255.50    NaN  13.49    Na

## Definition der Methoden und Metriken

#### Equally-Weighted Portfolio

In [4]:
#Rückgabe gleicher Gewichte anhand der Anzahl der Unternehmen (Input Returns)

def equally_weighted(returns):
    anzahl_assets = returns.shape[1]

    w0 = 1/anzahl_assets
    gewichte = np.full(anzahl_assets, w0)

    return gewichte

#### Minimum-Variance Portfolio

In [5]:
#Minimum-Variance Portfolio
#Code adaptiert von https://medium.com/@BorisGerat/mean-variance-and-minimum-variance-portfolio-models-in-python-64a5c6b57b2d

def min_variance_opt(matrix):
    
    #Compute the minimum-variance portfolio weights subject to the constraints of:
    #- No short-selling (weights must be >= 0)
    #- Full investment (sum of weights must equal 1)
    
    number_of_assets = matrix.shape[0]

    # Define the objective function to minimize portfolio variance
    def objective(w):
        portfolio_variance = np.dot(w, matrix @ w)
        return portfolio_variance

    # Constraint: The sum of portfolio weights must be equal to 1
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]

    # Bounds: Each weight must be between 0 and 1 (no short-selling)
    bounds = [[0, 1] for _ in range(number_of_assets)]

    # Initial guess: Equal weights for all assets
    w0 = np.ones(number_of_assets) / number_of_assets

    # Solve the optimization problem using Sequential Least Squares Quadratic Programming (SLSQP)
    res = minimize(objective, w0, method='SLSQP', bounds=bounds, constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    
    return res.x  # Return the optimal weights

#### Maximum Diversification Portfolio

In [6]:
#Code von https://thequantmba.wordpress.com/2017/06/06/max-diversification-in-python/ adaptiert

def calc_max_diversification_ratio(weights, matrix):
    
    # gewichtete Volatilität
    average_vol = np.dot(np.sqrt(np.diag(matrix)), weights.T)
    
    # portfolio volatilität
    portfolio_variance = np.dot(weights, matrix @ weights)
    portfolio_vol = np.sqrt(portfolio_variance)
    
    #Bestimmung Ratio
    diversification_ratio = average_vol/portfolio_vol
    
    # Negativer Wert für Minimierung (Maximizieren = Minimieren -)
    return -diversification_ratio

In [7]:
#Code von https://thequantmba.wordpress.com/2017/06/06/max-diversification-in-python/ adaptiert

def max_diversification_portfolio(matrix):
    
    anzahl_assets = matrix.shape[0]
    
    w0 = 1/anzahl_assets
    w = np.full(anzahl_assets, w0)
    
    # Constraint: The sum of portfolio weights must be equal to 1
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    
    # Bounds: Each weight must be between 0 and 1 (no short-selling)
    bounds = [[0, 1] for _ in range(anzahl_assets)]
    
    res = minimize(calc_max_diversification_ratio, w, bounds=bounds, args=(matrix), method='SLSQP', constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    return res.x

#### Inverse Volatility Portfolio

In [8]:
#Code adaptiert von https://www.kaggle.com/code/vijipai/lesson-4-traditional-portfolio-construction-method

def inverse_volatility_portfolio (matrix):
    
    standardabweichungen = np.sqrt(np.diagonal(matrix))
    invertierte_std = 1/standardabweichungen
    summe_invertierte_std = np.sum(invertierte_std)
    
    inverse_volatility_gewichte = invertierte_std / summe_invertierte_std
    
    return inverse_volatility_gewichte

#### Equal Risk Contribution Portfolio

In [9]:
#Code adaptiert von https://medium.com/@deepml1818/building-a-python-based-risk-parity-portfolio-for-trading-40441ecdd84d

def calculate_portfolio_volatility(weights, matrix):
    
    portfolio_variance = np.dot(weights, matrix @ weights)
    portfolio_volatility = np.sqrt(portfolio_variance)
    return portfolio_volatility

def calculate_risk_contribution(weights, matrix):
    portfolio_volatility = calculate_portfolio_volatility(weights, matrix)
    marginal_contribution_zaehler = np.dot(matrix, weights)
    risk_contribution = np.multiply(weights, marginal_contribution_zaehler) / portfolio_volatility
    return risk_contribution

def risk_parity_objective(weights, matrix):
    risk_contribution = calculate_risk_contribution(weights, matrix)
    target_risk = np.mean(risk_contribution)
    return np.sum((risk_contribution - target_risk) ** 2)

In [10]:
def get_risk_parity_weights(matrix):
    
    number_of_assets = matrix.shape[0]
    w0 = 1/number_of_assets
    initial_weights = np.full(number_of_assets, w0)
    
    constraints = ({'type': 'eq', 'fun': lambda initial_weights: np.sum(initial_weights) - 1})
    bounds = [[0, 1] for _ in range(number_of_assets)]
    
    result = minimize(risk_parity_objective, initial_weights, args=(matrix), method='SLSQP', bounds=bounds, constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    
    return result.x

#### Hierarchical Risk Parity

In [11]:
#Aus dem Paper von Lopez De Prado (2016)
def getIVP(cov,**kargs): # Compute the inverse-variance portfolio 
    ivp=1./np.diag(cov) 
    ivp/=ivp.sum() 
    return ivp

In [12]:
#------------------------------------------------------------------------------ 
def getClusterVar(cov,cItems): # Compute variance per cluster 
    cov_=cov.loc[cItems,cItems] # matrix slice 
    w_=getIVP(cov_).reshape(-1,1) 
    cVar=np.dot(np.dot(w_.T,cov_),w_)[0,0] 
    return cVar 

In [13]:
#------------------------------------------------------------------------------ 
def getQuasiDiag(link): # Sort clustered items by distance 
    link=link.astype(int) 
    sortIx=pd.Series([link[-1,0],link[-1,1]]) 
    numItems=link[-1,3] # number of original items 
    while sortIx.max()>=numItems: 
        sortIx.index=range(0,sortIx.shape[0]*2,2) # make space 
        df0=sortIx[sortIx>=numItems] # find clusters
        i=df0.index;j=df0.values-numItems 
        sortIx[i]=link[j,0] # item 1 
        df0=pd.Series(link[j,1],index=i+1) 
        sortIx = pd.concat([sortIx, df0]) # item 2  ####Anpassung wegen neuer Pandas-Version!
        sortIx=sortIx.sort_index() # re-sort 
        sortIx.index=range(sortIx.shape[0]) # re-index 
    return sortIx.tolist() 

In [14]:
#------------------------------------------------------------------------------ 
def getRecBipart(cov,sortIx): # Compute HRP allocation
    w = pd.Series(1,index=sortIx) 
    cItems=[sortIx] # initialize all items in one cluster 
    while len(cItems)>0: 
        #Bi-Section der Cluster
        cItems = [i[j:k] 
                  for i in cItems 
                  for j, k in ((0, len(i)//2), (len(i)//2, len(i))) 
                  if len(i) > 1]
        for i in range(0, len(cItems), 2): # parse in pairs 
            cItems0=cItems[i] # cluster 1 
            cItems1=cItems[i+1] # cluster 2 
            cVar0=getClusterVar(cov,cItems0) 
            cVar1=getClusterVar(cov,cItems1) 
            alpha=1-cVar0/(cVar0+cVar1) 
            w[cItems0]*=alpha # weight 1 
            w[cItems1]*=1-alpha # weight 2 
    return w 

In [15]:
#------------------------------------------------------------------------------ 
def correlDist(corr): 
    # A distance matrix based on correlation, where 0<=d[i,j]<=1 
    # This is a proper distance metric 
    dist=((1-corr)/2.)**.5 # distance matrix 
    
    #Adjustierung zur condensed_distance_matrix
    condensed_distance_matrix = squareform(X=dist, checks=False)
    
    return condensed_distance_matrix

In [16]:
def hrp_main(corr, cov, returns): 
    
    #3) cluster 
    dist=correlDist(corr) 
    link=sch.linkage(dist,'single') 
    sortIx=getQuasiDiag(link) 
    sortIx=corr.index[sortIx].tolist() # recover labels 
    df0=corr.loc[sortIx,sortIx] # reorder 
    
    #4) Capital allocation 
    hrp=getRecBipart(cov,sortIx) 
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    hrp_sortiert = hrp.reindex(original_reihenfolge)
    
    array = hrp_sortiert.T.values

    return array

#### Hierarchical Equal Risk Contribution Portfolio

In [17]:
#Es wird auf die Implementierung der Riskfolio-Lib zurückgegriffen
#Code adaptiert von #Code adaptiert von https://medium.com/@orenji.eirl/hierarchical-equal-risk-contribution-with-python-and-riskfolio-lib-ec45dd0f9899

def herc (returns):

    # Building the portfolio object
    port = hc(returns=returns)

    # Estimate optimal portfolio:

    model='HERC'
    correlation = 'pearson'
    rm = 'MV'
    rf = 0 
    linkage = 'ward' 
    max_k = 10 
    leaf_order = True 

    gewichte = port.optimization(model=model,
                          correlation=correlation,
                          rm=rm,
                          rf=rf,
                          linkage=linkage,
                          max_k=max_k,
                          leaf_order=leaf_order)
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    gewichte_sortiert = gewichte.reindex(original_reihenfolge)
    
    array = gewichte_sortiert.values.flatten()

    return array

#### Vergleichs-Metriken

In [18]:
def gesamt_return(gewichte, returns):

    daily_returns = returns @ gewichte
    total = np.prod(1 + daily_returns) - 1
    
    return total

In [19]:
def portfolio_return (gewichte, returns):
    
    avg_returns = []
    avg_returns = returns.mean()
    
    pf_return = sum([gewicht * rendite for gewicht, rendite in zip(gewichte, avg_returns)])
    
    return pf_return

In [20]:
def portfolio_variance (gewichte, returns):
    daily_returns = []
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    pf_var = np.var(daily_returns)
    
    return pf_var

In [21]:
def portfolio_volatility (gewichte, returns):
    variance = portfolio_variance(gewichte, returns)
    volatility = np.sqrt(variance)
    
    return volatility

In [22]:
def annualized_return(gewichte, returns):
    daily_returns = returns @ gewichte
    annual = (np.prod(1 + daily_returns))**(252/daily_returns.shape[0]) - 1
    
    return annual

In [23]:
def annualized_portfolio_return(gewichte, returns):
    pf_return = portfolio_return(gewichte, returns)
    
    return pf_return*252

In [24]:
def annualized_portfolio_volatility(gewichte, returns):
    pf_volatility = portfolio_volatility(gewichte, returns)
    
    return pf_volatility*np.sqrt(252)

In [25]:
def sharpe_ratio (gewichte, returns):
    pf_return = portfolio_return (gewichte, returns)
    volatility = portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [26]:
def sharpe_ratio_one (gewichte, returns):
    pf_return = portfolio_return (gewichte, returns)
    volatility = portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.01
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [27]:
def sharpe_ratio_two (gewichte, returns):
    pf_return = portfolio_return (gewichte, returns)
    volatility = portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.02
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [28]:
def annualized_sharpe_ratio (gewichte, returns):
    pf_return = annualized_portfolio_return (gewichte, returns)
    volatility = annualized_portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [29]:
def annualized_sharpe_ratio_one (gewichte, returns):
    pf_return = annualized_portfolio_return (gewichte, returns)
    volatility = annualized_portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.01
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [30]:
def annualized_sharpe_ratio_two (gewichte, returns):
    pf_return = annualized_portfolio_return (gewichte, returns)
    volatility = annualized_portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.02
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [31]:
def sortino_ratio (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [32]:
def sortino_ratio_one (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.01
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [33]:
def sortino_ratio_two (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.02
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [34]:
def annualized_sortino_ratio (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = annualized_portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [35]:
def annualized_sortino_ratio_one (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.01
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = annualized_portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [36]:
def annualized_sortino_ratio_two (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.02
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = annualized_portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [37]:
def maximum_drawdown (gewichte, returns):
    daily_returns = returns @ gewichte
    returns_kumuliert = (1 + daily_returns).cumprod()
    hochpunkte = returns_kumuliert.cummax()
    
    drawdowns = (returns_kumuliert - hochpunkte) / hochpunkte
    max_drawdown = drawdowns.min()
    
    #print(drawdowns)
    return max_drawdown

In [38]:
def calmar_ratio (gewichte, returns):
    pf_return = portfolio_return(gewichte, returns)
    max_drawdown = maximum_drawdown(gewichte, returns)
    
    calmar_ratio = pf_return / (-max_drawdown)
    
    return calmar_ratio

In [39]:
def value_at_risk_func(gewichte, returns):
    daily_returns = returns @ gewichte
    sorted_daily_returns = daily_returns.sort_values(ascending = True)
    value_at_risk = np.percentile(sorted_daily_returns, 100*0.05)
    return value_at_risk

In [40]:
#https://github.com/malctaylor15/VaR-and-Expected-Shortfall-Python/blob/master/Expected%20Shortfall%20and%20Value%20at%20Risk.ipynb

def expected_shortfall(gewichte, returns):
    daily_returns = returns @ gewichte
    sorted_daily_returns = daily_returns.sort_values(ascending = True)
    value_at_risk = value_at_risk_func(gewichte, returns)
    
    expected_shortfall = daily_returns[daily_returns <= value_at_risk].mean()
    
    return expected_shortfall

#### Sammel-Funktionen zum Berechnen und Speichern der Metriken

In [41]:
def metriken_equally_weighted(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile1 = {"Methode": "Equally-Weighted",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile1[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile1])], ignore_index=True)

In [42]:
def metriken_minimum_variance(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile2 = {"Methode": "Minimum_Variance",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile2[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile2])], ignore_index=True)

In [43]:
def metriken_max_diversification(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile3 = {"Methode": "Max-Diversification",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile3[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile3])], ignore_index=True)

In [44]:
def metriken_inverse_volatility(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile4 = {"Methode": "Inverse-Volatility",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile4[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile4])], ignore_index=True)

In [45]:
def metriken_risk_parity(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile5 = {"Methode": "Risk-Parity",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile5[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile5])], ignore_index=True)

In [46]:
def metriken_hrp(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile6 = {"Methode": "HRP",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile6[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile6])], ignore_index=True)

In [47]:
def metriken_herc(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile7 = {"Methode": "HERC",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile7[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile7])], ignore_index=True)

## Erzeugung Ergebnisse S&P500

## Ergebnis-Erzeugung JSE

In [48]:
returns = df_JSE.pct_change()
returns = returns.where(~df_JSE.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                 8303     15604     15618     16179     16183     16291  \
Date                                                                     
2005-06-02 -0.005128 -0.001771  0.006711  0.000690 -0.010465 -0.004525   
2005-06-03 -0.023110 -0.019516  0.008533 -0.000689 -0.001175 -0.006818   
2005-06-06 -0.023657 -0.000658  0.037678 -0.044828  0.014118  0.002288   
2005-06-07 -0.024230  0.002469 -0.007517  0.002888  0.005800  0.011416   
2005-06-08 -0.027970 -0.003284 -0.005135  0.006479  0.014994  0.006772   
...              ...       ...       ...       ...       ...       ...   
2025-05-26       NaN       NaN  0.013928       NaN -0.001542       NaN   
2025-05-27       NaN       NaN -0.003930       NaN  0.010039       NaN   
2025-05-28       NaN       NaN  0.004102       NaN  0.001529       NaN   
2025-05-29       NaN       NaN  0.010933       NaN  0.016794       NaN   
2025-05-30       NaN       NaN -0.016665       NaN  0.012763       NaN   

     

In [49]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 36
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    #Inf-Werte in NaN umwandeln
    training_daten = training_daten.replace([np.inf, -np.inf], np.nan)
    #Spalten entfernen, die nachher noch NaN enthalten
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 189
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = training_daten.cov()
    corr_matrix = training_daten.corr()
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    JSE_gewichte_equally_weighted = equally_weighted(training_daten)
    JSE_gewichte_min_variance= min_variance_opt(cov_matrix)
    JSE_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    JSE_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    JSE_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    JSE_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    JSE_gewichte_herc = herc(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": JSE_gewichte_equally_weighted,
    "Minimum-Variance": JSE_gewichte_min_variance,
    "Maximum-Diversification": JSE_gewichte_max_diversification,
    "Inverse-Volatility": JSE_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": JSE_gewichte_risk_parity,
    "HRP": JSE_gewichte_hrp,
    "HERC": JSE_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
# Alle Testperioden zusammenführen
portfolio_returns_df_JSE = pd.concat(portfolio_returns_list)
portfolio_returns_df_JSE = portfolio_returns_df_JSE.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.140247654123246e-05
            Iterations: 214
            Function evaluations: 19689
            Gradient evaluations: 214
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.6107736720313213
            Iterations: 60
            Function evaluations: 5579
            Gradient evaluations: 60
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.433367165256956e-09
            Iterations: 56
            Function evaluations: 5152
            Gradient evaluations: 56
2005-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.207062012568762e-05
            Iterations: 233
            Function evaluations: 21903
            Gradient evaluations: 233
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.664

Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.7619569346564123
            Iterations: 66
            Function evaluations: 6402
            Gradient evaluations: 66
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.01570418142266e-08
            Iterations: 64
            Function evaluations: 6145
            Gradient evaluations: 64
2006-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.592713324620554e-05
            Iterations: 191
            Function evaluations: 18145
            Gradient evaluations: 191
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.856604967040928
            Iterations: 68
            Function evaluations: 6528
            Gradient evaluations: 68
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.924005978097399e-08
           

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0836681786641476e-08
            Iterations: 123
            Function evaluations: 11193
            Gradient evaluations: 123
2007-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.5235567928049415e-05
            Iterations: 201
            Function evaluations: 18292
            Gradient evaluations: 201
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.7778584876837247
            Iterations: 57
            Function evaluations: 5245
            Gradient evaluations: 57
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0788360795860455e-08
            Iterations: 122
            Function evaluations: 11103
            Gradient evaluations: 122
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 

2008-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.070168514039482e-05
            Iterations: 214
            Function evaluations: 19046
            Gradient evaluations: 214
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.7905597046286092
            Iterations: 62
            Function evaluations: 5576
            Gradient evaluations: 62
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.9630883671983283e-08
            Iterations: 65
            Function evaluations: 5786
            Gradient evaluations: 65
2008-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8657764474230258e-05
            Iterations: 194
            Function evaluations: 17267
            Gradient evaluations: 194
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.8

Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.9140345174557423
            Iterations: 57
            Function evaluations: 5302
            Gradient evaluations: 57
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0477270372399053e-08
            Iterations: 45
            Function evaluations: 4140
            Gradient evaluations: 45
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1787096928903132e-05
            Iterations: 233
            Function evaluations: 21203
            Gradient evaluations: 233
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.9262450179278274
            Iterations: 59
            Function evaluations: 5433
            Gradient evaluations: 59
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0035234851429239e-08
      

...

2017-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.2203775615551524e-05
            Iterations: 190
            Function evaluations: 21660
            Gradient evaluations: 190
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.815234455210036
            Iterations: 62
            Function evaluations: 7131
            Gradient evaluations: 62
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1963412751291415e-09
            Iterations: 63
            Function evaluations: 7182
            Gradient evaluations: 63
2017-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.369939192474124e-05
            Iterations: 233
            Function evaluations: 26796
            Gradient evaluations: 233
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.79

Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.526694036516541
            Iterations: 71
            Function evaluations: 8609
            Gradient evaluations: 71
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.2476544145901396e-08
            Iterations: 66
            Function evaluations: 7921
            Gradient evaluations: 66
2018-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.2829869227690657e-05
            Iterations: 287
            Function evaluations: 34441
            Gradient evaluations: 287
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.513425471720502
            Iterations: 74
            Function evaluations: 8975
            Gradient evaluations: 74
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.2173330884391698e-08
        

Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.5468702618614597e-08
            Iterations: 76
            Function evaluations: 9120
            Gradient evaluations: 76
2019-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8688574386126337e-05
            Iterations: 288
            Function evaluations: 34848
            Gradient evaluations: 288
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.546813646810357
            Iterations: 81
            Function evaluations: 9787
            Gradient evaluations: 80
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.4646807993487657e-08
            Iterations: 77
            Function evaluations: 9317
            Gradient evaluations: 77
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.21554

2020-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.143122674559335e-06
            Iterations: 405
            Function evaluations: 49815
            Gradient evaluations: 405
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.168110164706431
            Iterations: 142
            Function evaluations: 17559
            Gradient evaluations: 138
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.9277135670818477e-08
            Iterations: 44
            Function evaluations: 5413
            Gradient evaluations: 44
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.949902864827433e-06
            Iterations: 365
            Function evaluations: 44530
            Gradient evaluations: 365
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.

Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.1812416782298945
            Iterations: 94
            Function evaluations: 12224
            Gradient evaluations: 93
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.736073659947652e-08
            Iterations: 43
            Function evaluations: 5590
            Gradient evaluations: 43
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.077750878389717e-06
            Iterations: 415
            Function evaluations: 54366
            Gradient evaluations: 415
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.177085650073752
            Iterations: 95
            Function evaluations: 12450
            Gradient evaluations: 94
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.631893807211175e-08
        

In [50]:
#Exportieren nach CSV
portfolio_returns_df_JSE.to_csv("returnsWharton_neu_3jahre_f.csv")

## Ledoit-Wolf Shrinkage

In [51]:
#Definition von Ledoit-Shrinkage

def ledoit_shrink (returns):
    
    shrinkage = LedoitWolf().fit(returns.values)
    
    shrinkage_matrix = pd.DataFrame(
        shrinkage.covariance_,
        index = returns.columns,
        columns = returns.columns)
    
    #print(shrinkage.shrinkage_)
    
    return shrinkage_matrix

### Auswertung S&P500 mit Ledoit Wolf

### Auswertung JSE mit Ledoit Wolf

In [52]:
returns = df_JSE.pct_change()
returns = returns.where(~df_JSE.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                 8303     15604     15618     16179     16183     16291  \
Date                                                                     
2005-06-02 -0.005128 -0.001771  0.006711  0.000690 -0.010465 -0.004525   
2005-06-03 -0.023110 -0.019516  0.008533 -0.000689 -0.001175 -0.006818   
2005-06-06 -0.023657 -0.000658  0.037678 -0.044828  0.014118  0.002288   
2005-06-07 -0.024230  0.002469 -0.007517  0.002888  0.005800  0.011416   
2005-06-08 -0.027970 -0.003284 -0.005135  0.006479  0.014994  0.006772   
...              ...       ...       ...       ...       ...       ...   
2025-05-26       NaN       NaN  0.013928       NaN -0.001542       NaN   
2025-05-27       NaN       NaN -0.003930       NaN  0.010039       NaN   
2025-05-28       NaN       NaN  0.004102       NaN  0.001529       NaN   
2025-05-29       NaN       NaN  0.010933       NaN  0.016794       NaN   
2025-05-30       NaN       NaN -0.016665       NaN  0.012763       NaN   

     

In [53]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 36
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
     #Inf-Werte in NaN umwandeln
    training_daten = training_daten.replace([np.inf, -np.inf], np.nan)
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 189
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = ledoit_shrink(training_daten)
    
    #aus der geschrumpften Matrix die Korrelationsmatrix bauen
    standardabweichung = np.sqrt(np.diag(cov_matrix))
    corr_matrix = cov_matrix / np.outer(standardabweichung, standardabweichung)
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    shrinked_JSE_gewichte_equally_weighted = equally_weighted(training_daten)
    shrinked_JSE_gewichte_min_variance= min_variance_opt(cov_matrix)
    shrinked_JSE_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    shrinked_JSE_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    shrinked_JSE_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    shrinked_JSE_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    shrinked_JSE_gewichte_herc = herc(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": shrinked_JSE_gewichte_equally_weighted,
    "Minimum-Variance": shrinked_JSE_gewichte_min_variance,
    "Maximum-Diversification": shrinked_JSE_gewichte_max_diversification,
    "Inverse-Volatility": shrinked_JSE_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": shrinked_JSE_gewichte_risk_parity,
    "HRP": shrinked_JSE_gewichte_hrp,
    "HERC": shrinked_JSE_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
#Alle Testperioden zusammenführen
portfolio_returns_df_shrinked_JSE = pd.concat(portfolio_returns_list)
portfolio_returns_df_shrinked_JSE = portfolio_returns_df_shrinked_JSE.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.3370717934394173e-05
            Iterations: 172
            Function evaluations: 15824
            Gradient evaluations: 172
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.7093681458752172
            Iterations: 65
            Function evaluations: 6045
            Gradient evaluations: 65
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0126676189220253e-08
            Iterations: 41
            Function evaluations: 3772
            Gradient evaluations: 41
2005-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.419697595145844e-05
            Iterations: 195
            Function evaluations: 18330
            Gradient evaluations: 195
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.7

Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.822902175166995
            Iterations: 67
            Function evaluations: 6498
            Gradient evaluations: 67
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.70694265810574e-08
            Iterations: 76
            Function evaluations: 7296
            Gradient evaluations: 76
2006-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.859336722614699e-05
            Iterations: 185
            Function evaluations: 17575
            Gradient evaluations: 185
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.9189283526408087
            Iterations: 72
            Function evaluations: 6922
            Gradient evaluations: 72
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.7341121836785657e-08
          

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0840218671325276e-08
            Iterations: 112
            Function evaluations: 10192
            Gradient evaluations: 112
2007-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.7396853545887523e-05
            Iterations: 179
            Function evaluations: 16289
            Gradient evaluations: 179
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.811521535484661
            Iterations: 63
            Function evaluations: 5796
            Gradient evaluations: 63
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.083958923393602e-08
            Iterations: 112
            Function evaluations: 10192
            Gradient evaluations: 112
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.

2008-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.2097166763145687e-05
            Iterations: 188
            Function evaluations: 16732
            Gradient evaluations: 188
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.811585772941292
            Iterations: 63
            Function evaluations: 5673
            Gradient evaluations: 63
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6960256591147298e-08
            Iterations: 77
            Function evaluations: 6853
            Gradient evaluations: 77
2008-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.008922165344392e-05
            Iterations: 187
            Function evaluations: 16643
            Gradient evaluations: 187
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.88

Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.957540228964773
            Iterations: 63
            Function evaluations: 5865
            Gradient evaluations: 63
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.864665106648274e-09
            Iterations: 45
            Function evaluations: 4140
            Gradient evaluations: 45
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.192163490223978e-05
            Iterations: 209
            Function evaluations: 19019
            Gradient evaluations: 209
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.967585665962645
            Iterations: 64
            Function evaluations: 5891
            Gradient evaluations: 64
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.477435806032482e-09
           

...

2017-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.402831105803544e-05
            Iterations: 178
            Function evaluations: 20292
            Gradient evaluations: 178
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.977534578387774
            Iterations: 71
            Function evaluations: 8164
            Gradient evaluations: 71
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.7289867534268757e-09
            Iterations: 52
            Function evaluations: 5929
            Gradient evaluations: 52
2017-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.470941389550162e-05
            Iterations: 159
            Function evaluations: 18286
            Gradient evaluations: 159
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.959

Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.530522937029159
            Iterations: 82
            Function evaluations: 9945
            Gradient evaluations: 82
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.035349737970589e-08
            Iterations: 68
            Function evaluations: 8161
            Gradient evaluations: 68
2018-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.636932017143883e-05
            Iterations: 187
            Function evaluations: 22441
            Gradient evaluations: 187
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.524395092347816
            Iterations: 153
            Function evaluations: 18539
            Gradient evaluations: 149
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.94242210483327e-08
         

Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.3069050885471245e-08
            Iterations: 78
            Function evaluations: 9361
            Gradient evaluations: 78
2019-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.4868384182988905e-05
            Iterations: 215
            Function evaluations: 26016
            Gradient evaluations: 215
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.544538613924099
            Iterations: 84
            Function evaluations: 10268
            Gradient evaluations: 84
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.224870753416063e-08
            Iterations: 78
            Function evaluations: 9438
            Gradient evaluations: 78
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.06436

2020-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2046807795730703e-05
            Iterations: 261
            Function evaluations: 32104
            Gradient evaluations: 261
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.246931296347366
            Iterations: 93
            Function evaluations: 11573
            Gradient evaluations: 93
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.7493281461975763e-08
            Iterations: 44
            Function evaluations: 5413
            Gradient evaluations: 44
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1675814238834481e-05
            Iterations: 281
            Function evaluations: 34282
            Gradient evaluations: 281
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.

Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.241373219641934
            Iterations: 92
            Function evaluations: 12095
            Gradient evaluations: 92
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.5919638409165734e-08
            Iterations: 44
            Function evaluations: 5720
            Gradient evaluations: 44
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0929315004488973e-05
            Iterations: 297
            Function evaluations: 38908
            Gradient evaluations: 297
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.2389263006399265
            Iterations: 92
            Function evaluations: 12182
            Gradient evaluations: 92
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4957668204951477e-08
     

In [54]:
#Exportieren nach CSV
portfolio_returns_df_shrinked_JSE.to_csv("returnsWhartonshrinked_neu_3jahre_f.csv")

### Reduzierter S&P500-Bestand

### Reduziert und Ledoit-Shrinkage